In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import tensorflow as tf


In [ ]:
url="https://raw.githubusercontent.com/Sooryak12/FakeReviewDetectionLSTM/master/selected.txt"

In [ ]:
df=pd.read_csv(url,delimiter="\t")
df.head()

In [ ]:
df["LABEL"]=df["LABEL"].replace(['__label2__','__label1__'],['0','1'])
dfs=df[["REVIEW_TEXT","LABEL","PRODUCT_CATEGORY"]]
dfs = dfs.rename(columns={'REVIEW_TEXT': 'reviews',"LABEL":"label","PRODUCT_CATEGORY":"cat"})
dfs["label"]=dfs["label"].astype(np.float32)

In [ ]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)
dfs["reviews"]=dfs["reviews"].apply(preprocess_text)
dfs["reviews"]=dfs["reviews"].apply(remove_tags)
dfse["reviews"]=dfse["reviews"].str.lower()

In [ ]:
from nlppreprocess import NLP
obj=NLP(remove_stopwords=True,remove_numbers=True)
dfse["reviews"]=dfse["reviews"].apply(obj.process)

In [ ]:
dfs["len_review"]=dfs["reviews"].apply(lambda x :len(x.split()))
dfse=dfs[dfs["len_review"]<150]
dfse.reset_index(drop=True,inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Ytrain,Ytest = train_test_split(dfse["reviews"],dfse["label"], test_size=0.2, random_state=43)

In [ ]:
from transformers import BertTokenizer


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [ ]:
dfse['data_type'] = ['not_set']*dfse.shape[0]

In [ ]:
dfse.loc[df['id'].isin(X_train),'data_type']= 'train'
dfse.loc[df['id'].isin(X_test),'data_type']= 'test'

In [ ]:
###bert_input = tokenizer.encode_plus(
                        test_sentence,                      
                        add_special_tokens = True, # add [CLS], [SEP]
                        max_length = max_length_test, # max length of the text that can go to BERT
                        pad_to_max_length = True, # add [PAD] tokens
                        return_attention_mask = True, # add attention mask to not focus on pad tokens
              )
print('encoded', bert_input)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    dfse[dfse.data_type=='train'].reviews.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='tf'
)


encoded_data_test = tokenizer.batch_encode_plus(
    dfse[dfse.data_type=='train'].reviews.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='tf'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = (dfse[dfse.data_type=='train'].label.values)


input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = dfse[dfse.data_type=='test'].label.values


In [ ]:
dataset_train = tf.data.Dataset.from_tensor_slices(input_ids_train, attention_masks_train, labels_train)

dataset_test = tf.data.Dataset.from_tensor_slices(input_ids_test, attention_masks_test, labels_test)

In [ ]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf

# recommended learning rate for Adam 5e-5, 3e-5, 2e-5
learning_rate = 2e-5
# we will do just 1 epoch for illustration, though multiple epochs might be better as long as we will not overfit the model
number_of_epochs = 5




In [ ]:

# model initialization
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', 
                                                        num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)


In [ ]:
# optimizer Adam recommended
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)

# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.BinaryCrossCrossentropy(from_logits=True)
metric = tf.keras.metrics.BinaryAccuracy()
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
r= model.fit(dataset_train, epochs=number_of_epochs, validation_data=dataset_test)